In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random

In [117]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)

url = 'https://www.aruodas.lt/butai/vilniuje/puslapis/2/?FOwnerDbId0=1&FOwnerDbId2=1' 
driver.get(url)
time.sleep(2)
source = driver.page_source
bs = BeautifulSoup(source, 'html.parser')
aruodopirmas = bs.find_all('div', {'class':'advert-flex'})

nuorodos=[]
plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas = [], [], [], [], [], [], [], []
energetineklase, ilguma, platuma = [], [], []

for aruodpirm in aruodopirmas:
    try:
        nuorodos.append(aruodpirm.find('a')['href'])
        url1 = nuorodos[-1] 
        driver.get(url1)
        time.sleep(2)
        source1 = driver.page_source
        bs1 = BeautifulSoup(source1, 'html.parser')
    except:
        pass
    else:
        asas=bs1.find('dl', {'class': 'obj-details'})
        # print(asas.prettify())

        bandymas=asas.find_all('dt')
        band=asas.find_all('dd')
        bandymas=[i.text.strip() for i in bandymas]
        band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
        band2= [i.text.strip() if i is not None else str(None) for i in band]
        plotas.append(None)
        kambariusk.append(None)
        aukstas.append(None)
        aukstusk.append(None)
        metai.append(None)
        pastatotipas.append(None)
        sildymas.append(None)
        irengimas.append(None)
        energetineklase.append(None)
        ilguma.append(None)
        platuma.append(None)

        for p, k in zip(bandymas, band2):
            if p == 'Plotas:':
                plotas[-1]=k
            if p == 'Kambarių sk.:':
                kambariusk[-1]=k
            if p == 'Aukštas:':
                aukstas[-1]=k
            if p == 'Aukštų sk.:':
                aukstusk[-1]=k
            if p == 'Metai:':
                metai[-1]=k
            if p == 'Pastato tipas:':
                pastatotipas[-1]=k
            if p == 'Šildymas:':
                sildymas[-1]=k
            if p == 'Įrengimas:':
                irengimas[-1]=k
            if p == 'Pastato energijos suvartojimo klasė:':
                energetineklase[-1]=k
            
            try:    
                tim=bs1.find('a', {'class':'link-obj-thumb vector-thumb-map'})
                xy=str(tim['href']).split('=')[-1].split('%2C')
            except:
                pass
            else:
                ilguma[-1]=xy[0]
                platuma[-1]=xy[1]
        
    
    # if aruodpirm == aruodopirmas[4]:
    #     break
    
driver.close()
    

In [120]:
print(ilguma)
print(platuma)

['54.672208', '54.659689', '54.659938', '54.717506', '54.70066', '54.715166', '54.691778', '54.691778', '54.682749', '54.728983', '54.710658', '54.693295', '54.727032', '54.720741', '54.742081', '54.690782', '54.740417', '54.718771', '54.718898', '54.718898', '54.725559', '54.715901', '54.699206', '54.702581', '54.715901']
['25.208113', '25.18961', '25.188841', '25.239784', '25.247741', '25.176891', '25.293523', '25.293523', '25.281340', '25.221034', '25.183349', '25.294861', '25.282146', '25.249978', '25.274941', '25.264339', '25.233992', '25.269841', '25.270229', '25.270229', '25.245515', '25.281605', '25.316907', '25.281505', '25.281605']


In [91]:
plotas, kambariusk, aukstas, aukstusk, metai, pastatotipas, sildymas, irengimas = [], [], [], [], [], [], [], []
energetineklase = []

asas=bs1.find('dl', {'class': 'obj-details'})
# print(asas.prettify())

bandymas=asas.find_all('dt')
band=asas.find_all('dd')
bandymas=[i.text.strip() for i in bandymas]
band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
band2= [i.text.strip() if i is not None else str(None) for i in band]
plotas.append(None)
kambariusk.append(None)
aukstas.append(None)
aukstusk.append(None)
metai.append(None)
pastatotipas.append(None)
sildymas.append(None)
irengimas.append(None)
energetineklase.append(None)

for p, k in zip(bandymas, band2):
    if p == 'Plotas:':
        plotas[-1]=k
    if p == 'Kambarių sk.:':
        kambariusk[-1]=k
    if p == 'Aukštas:':
        aukstas[-1]=k
    if p == 'Aukštų sk.:':
        aukstusk[-1]=k
    if p == 'Metai:':
        metai[-1]=k
    if p == 'Pastato tipas:':
        pastatotipas[-1]=k
    if p == 'Šildymas:':
        sildymas[-1]=k
    if p == 'Įrengimas:':
        irengimas[-1]=k
    if p == 'Pastato energijos suvartojimo klasė:':
        energetineklase[-1]=k
    print(p, k)
    
print(metai)


Namo numeris: 34
Plotas: 63,09 m²
Kambarių sk.: 3
Aukštas: 2
Aukštų sk.: 3
Metai: 2023
Pastato tipas: Monolitinis
Šildymas: Geoterminis
Įrengimas: Dalinė apdaila
Pastato energijos suvartojimo klasė: A++
Ypatybės: None
Papildomos patalpos: None
Papildoma įranga: None
Apsauga: None
['2023']


In [77]:
bandymas=asas.find_all('dt')
band=asas.find_all('dd')

bandymas=[i.text for i in bandymas]
band=[i.find('span', {'class': 'fieldValueContainer'}) for i in band]
band2= [i.text if i is not None else str(None) for i in band]
for p, k in zip(bandymas, band2):
    print(p.strip(), k.strip())

Namo numeris: 34
Plotas: 63,09 m²
Kambarių sk.: 3
Aukštas: 2
Aukštų sk.: 3
Metai: 2023
Pastato tipas: Monolitinis
Šildymas: Geoterminis
Įrengimas: Dalinė apdaila
Pastato energijos suvartojimo klasė: A++
Ypatybės: None
Papildomos patalpos: None
Papildoma įranga: None
Apsauga: None
